In [3]:
#pip install openmeteo-requests
#!pip install requests-cache retry-requests numpy pandas

import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/dwd-icon"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation", "wind_speed_10m"],
	"timezone": "Europe/London",
	"past_days": 31
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(5).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["precipitation"] = hourly_precipitation
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)

Coordinates 52.52000045776367°N 13.419998168945312°E
Elevation 38.0 m asl
Timezone b'Europe/London' b'BST'
Timezone difference to GMT+0 3600 s
                         date  temperature_2m  relative_humidity_2m   
0   2024-05-25 23:00:00+00:00       16.595499                  82.0  \
1   2024-05-26 00:00:00+00:00       15.595500                  91.0   
2   2024-05-26 01:00:00+00:00       14.795500                  97.0   
3   2024-05-26 02:00:00+00:00       14.095500                  95.0   
4   2024-05-26 03:00:00+00:00       13.695499                  93.0   
..                        ...             ...                   ...   
907 2024-07-02 18:00:00+00:00       19.069500                  51.0   
908 2024-07-02 19:00:00+00:00       18.069500                  56.0   
909 2024-07-02 20:00:00+00:00       16.869501                  63.0   
910 2024-07-02 21:00:00+00:00       15.769500                  69.0   
911 2024-07-02 22:00:00+00:00       14.969500                  73.0   

    

In [4]:
hourly_dataframe

,date,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,wind_speed_10m
0,2024-05-25 23:00:00+00:00,16.595499,82.0,13.509601,16.902561,0.0,5.623380
1,2024-05-26 00:00:00+00:00,15.595500,91.0,14.131086,16.396255,0.0,3.758510
2,2024-05-26 01:00:00+00:00,14.795500,97.0,14.323655,15.524284,0.0,4.735060
3,2024-05-26 02:00:00+00:00,14.095500,95.0,13.306198,14.558578,0.0,4.072935
4,2024-05-26 03:00:00+00:00,13.695499,93.0,12.583660,14.175747,0.0,2.276840
...,...,...,...,...,...,...,...
907,2024-07-02 18:00:00+00:00,19.069500,51.0,8.697532,17.274546,0.0,9.957108
908,2024-07-02 19:00:00+00:00,18.069500,56.0,9.156044,16.675785,0.0,8.049845
909,2024-07-02 20:00:00+00:00,16.869501,63.0,9.781803,15.876669,0.0,6.489992
910,2024-07-02 21:00:00+00:00,15.769500,69.0,10.094409,15.069830,0.0,5.091168
